In [1]:
from datetime import datetime

class Stats:
    def __init__(self, command: str, id: int):
        self.command = command
        self.response = None
        self.id = id

        self.start_time = datetime.now()
        self.end_time = None
        self.duration = None

    def add_response(self, response: str):
        self.response = str(response)
        # Calculating total time taken to execute command
        self.end_time = datetime.now()
        self.duration = (self.end_time-self.start_time).total_seconds()

    def got_response(self):
        if self.response is None:
            return False
        else:
            return True

    def get_raw_response(self):
        return self.response

    def numeric_response(self, data: str):
        num_val = ''.join(i for i in data if i.isdigit() or i=='-' or i=='.')
        return num_val

    def int_response(self, data: str):
        return int(self.numeric_response(data))

    def float_response(self, data: str):
        return float(self.numeric_response(data))
    
    def attitude_response(self):
        raw_att = self.response.split(';')
        att_data = (self.int_response(raw_att[0]), self.int_response(raw_att[1]), self.int_response(raw_att[2]))
        return att_data
    
    def acceleration_response(self):
        raw_acc = self.response.split(';')
        acc_data = (self.float_response(raw_acc[0]), self.float_response(raw_acc[1]), self.float_response(raw_acc[2]))
        return acc_data

    def temp_response(self):
        raw_temp = self.response.split('~')
        temp = (self.int_response(raw_temp[0]) + self.int_response(raw_temp[1]))/2
        return temp

    def get_response(self):
        if 'attitude?' in self.command:
            return self.attitude_response()
        elif 'acceleration?' in self.command:
            return self.acceleration_response()
        elif 'temp?' in self.command:
            return self.temp_response()
        elif 'baro?' in self.command or 'speed?' in self.command:
            return self.float_response(self.response)
        elif '?' not in self.command:
            return self.get_raw_response()
        else:
            return self.int_response(self.response)
        
        
        
        
        
        
        
import socket
import threading
import time
import cv2

class Tello:    
    def __init__(self,local_ip, local_port , tello_ip: str='192.168.10.1', debug: bool=True):
        # Opening local UDP port on 8889 for Tello communication
        self.local_ip = local_ip
        self.local_port = local_port
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind((self.local_ip, self.local_port))
        
        # Setting Tello ip and port info
        self.tello_ip = tello_ip
        self.tello_port = 8889
        self.tello_address = (self.tello_ip, self.tello_port)
        self.log = []

        # Intializing response thread
        self.receive_thread = threading.Thread(target=self._receive_thread)
        self.receive_thread.daemon = True
        self.receive_thread.start()

        # easyTello runtime options
        self.stream_state = False
        self.MAX_TIME_OUT = 15.0
        self.debug = debug
        # Setting Tello to command mode
        self.command()

    def send_command(self, command: str, query: bool =False):
        # New log entry created for the outbound command
        self.log.append(Stats(command, len(self.log)))

        # Sending command to Tello
        self.socket.sendto(command.encode('utf-8'), self.tello_address)
        # Displaying conformation message (if 'debug' os True)
        if self.debug is True:
            print('Sending command: {}'.format(command))
            
        # Checking whether the command has timed out or not (based on value in 'MAX_TIME_OUT')
        start = time.time()
        while not self.log[-1].got_response():  # Runs while no repsonse has been received in log
            now = time.time()
            difference = now - start
            if difference > self.MAX_TIME_OUT:
                print('Connection timed out!')
                break
        # Prints out Tello response (if 'debug' is True)
        if self.debug is True and query is False:
            print('Response: {}'.format(self.log[-1].get_response()))

    def _receive_thread(self):
        while True:
            # Checking for Tello response, throws socket error
            try:
                self.response, ip = self.socket.recvfrom(1024)
                self.log[-1].add_response(self.response)
            except socket.error as exc:
                print('Socket error: {}'.format(exc))

    def _video_thread(self):
        # Creating stream capture object
        cap = cv2.VideoCapture('udp://'+self.tello_ip+':11111')
        cv2.namedWindow(self.local_ip)

        # Runs while 'stream_state' is True
        while self.stream_state:
            ret, frame = cap.read()
            print (self.local_ip)
            cv2.imshow(self.local_ip , frame)
            # Video Stream is closed if escape key is pressed
            k = cv2.waitKey(1) & 0xFF
            if k == 27:
                break
        cap.release()
        cv2.destroyWindow(self.local_ip)    
        
    def wait(self, delay: float):
        # Displaying wait message (if 'debug' is True)
        if self.debug is True:
            print('Waiting {} seconds...'.format(delay))
        # Log entry for delay added
        self.log.append(Stats('wait', len(self.log)))
        # Delay is activated
        time.sleep(delay)
    
    def get_log(self):
        return self.log

    # Controll Commands
    def command(self):
        self.send_command('command')
    
    def takeoff(self):
        self.send_command('takeoff')

    def land(self):
        self.send_command('land')

    def streamon(self):
        self.send_command('streamon')
        self.stream_state = True
#         self.video_thread = threading.Thread(target=self._video_thread)
#         self.video_thread.daemon = True
#         self.video_thread.start()

    def streamoff(self):
        self.stream_state = False
        self.send_command('streamoff')

    def emergency(self):
        self.send_command('emergency')
    
    # Movement Commands
    def up(self, dist: int):
        self.send_command('up {}'.format(dist))

    def down(self, dist: int):
        self.send_command('down {}'.format(dist))

    def left(self, dist: int):
        self.send_command('left {}'.format(dist))

    def right(self, dist: int):
        self.send_command('right {}'.format(dist))
        
    def forward(self, dist: int):
        self.send_command('forward {}'.format(dist))

    def back(self, dist: int):
        self.send_command('back {}'.format(dist))

    def cw(self, degr: int):
        self.send_command('cw {}'.format(degr))
    
    def ccw(self, degr: int):
        self.send_command('ccw {}'.format(degr))

    def flip(self, direc: str):
        self.send_command('flip {}'.format(direc))

    def go(self, x: int, y: int, z: int, speed: int):
        self.send_command('go {} {} {} {}'.format(x, y, z, speed))

    def curve(self, x1: int, y1: int, z1: int, x2: int, y2: int, z2: int, speed: int):
        self.send_command('curve {} {} {} {} {} {} {}'.format(x1, y1, z1, x2, y2, z2, speed))

    # Set Commands
    def set_speed(self, speed: int):  
        self.send_command('speed {}'.format(speed))

    def rc_control(self, a: int, b: int, c: int, d: int):
        self.send_command('rc {} {} {} {}'.format(a, b, c, d))

    def set_wifi(self, ssid: str, passwrd: str):
        self.send_command('wifi {} {}'.format(ssid, passwrd))

    # Read Commands
    def get_speed(self):
        self.send_command('speed?', True)
        return self.log[-1].get_response()

    def get_battery(self):
        self.send_command('battery?', True)
        return self.log[-1].get_response()

    def get_time(self):
        self.send_command('time?', True)
        return self.log[-1].get_response()

    def get_height(self):
        self.send_command('height?', True)
        return self.log[-1].get_response()
    
    def get_temp(self):
        self.send_command('temp?', True)
        return self.log[-1].get_response()

    def get_attitude(self):
        self.send_command('attitude?', True)
        return self.log[-1].get_response()

    def get_baro(self):
        self.send_command('baro?', True)
        return self.log[-1].get_response()

    def get_acceleration(self):
        self.send_command('acceleration?', True)
        return self.log[-1].get_response()
    
    def get_tof(self):
        self.send_command('tof?', True)
        return self.log[-1].get_response()

    def get_wifi(self):
        self.send_command('wifi?', True)
        return self.log[-1].get_response()

In [2]:
telloDrone= Tello('192.168.10.2', 8889)
telloDrone.takeoff()

Sending command: command
Response: b'ok'
Sending command: takeoff
Response: b'ok'


In [ ]:
telloDrone.takeoff()

In [3]:
class telloTest:
    
    
    def __inti__(self):
        this.refX=0
        this.refY=0
        this.refZ=0
    
    def updateRefX(self,refX):
        #print("RefX:")
        #print(refX)
        self.refX=refX
    def updateRefY(self,refY):
        #print("RefY")
        #print(refY)
        self.refY=refY
    def updateRefZ(self,refZ):
        self.refZ=refZ
    def getRefX(self):
        return self.refX
    def getRefY(self):
        return self.refY
    def getRefZ(self):
        return self.refZ
    
   





    
    
import random
from socket import *
import time
tello= telloTest()
serverSocket = socket(AF_INET, SOCK_DGRAM)
serverSocket.bind(('', 8000))

referenceX=0
referenceY=0
referenceZ=0

newValueX=0
newValueY=0
newValueZ=0
data=[0,0,0]
check=0



while True:
    message, address = serverSocket.recvfrom(1024)
    message=message.decode("utf-8")
  

    data=message.split(",")  
    newValueX=float(data[1])
    newValueY=float(data[0])
    newValueZ=float(data[2])
    
    if check==0 and float(data[0])!=0 and float(data[1])!=0:
        print("setting refX",newValueX)
        print("setting refY",newValueY)
        print("setting refZ",newValueZ)
        
        tello.updateRefX(newValueX)
        tello.updateRefY(newValueY)
        tello.updateRefZ(newValueZ)
        check=1
        continue
        
    
    referenceX=tello.getRefX()
    referenceY=tello.getRefY()
    referenceZ=tello.getRefZ()
    
    
    #Translate the reference to 90 degrees, find X,Y,Z
    X = 90-referenceX
    Y = 90-referenceY
    Z = 90-referenceZ
    
    
    newValueX=newValueX+X
    newValueY=newValueY+Y
    newValueZ=newValueZ+Z
    
    if(newValueX<0):
        newValueX=newValueX+360
    elif(newValueX>360):
        newValueX=newValueX%360
    
    
    if(newValueY<0):
        newValueY=newValueY + 360
    elif(newValueY>360):
        newValueY=newValueY % 360
    
    
    if(newValueZ<0):
        newValueZ=newValueZ + 360
    elif(newValueZ>360):
        newValueZ=newValueZ % 360
    
    
    
    #Check the threshold
    
    
    
    if(newValueY<75 or newValueY>105):
        # Y = ClockWise and AntiClockWise Rotation
        if(newValueY<90):
            print("newValueY=",newValueY)
            print("AntiClockWise")
            #ClockWise
            rotate=90-newValueY
            telloDrone.ccw(rotate)
        elif(newValueY>90):
            print("newValueY=",newValueY)
            print("ClockWise")
            #AntiCLockWise
            rotate=newValueY-90
            telloDrone.cw(rotate)
    
    
    if(newValueX<75 or newValueX>105):
        # X = pitch
        if(newValueX<90):
            print("newValueX=",newValueX)
            print("backward")
            telloDrone.back(90)
        elif(newValueX>90):
            print("newValueX=",newValueX)
            print("forward")
            telloDrone.forward(90)
    
    if(newValueZ<75 or newValueZ>105):
        # Z = Side Ways
        if(newValueZ<90):
            
            print("newValueZ=",newValueZ)
            print("Right")
            telloDrone.right(90)

        elif(newValueZ>90):
            print("newValueZ=",newValueZ)
            print("Left")
            telloDrone.left(90)

    
    


setting refX 1.0
setting refY 297.0
setting refZ 356.0
newValueY= 110.0
ClockWise
Sending command: cw 20.0
Response: b'ok'
newValueX= 49.0
backward
Sending command: back 90
Response: b'ok'
newValueY= 109.0
ClockWise
Sending command: cw 19.0
Response: b'ok'
newValueZ= 115.0
Left
Sending command: left 90
Response: b'ok'
newValueY= 108.0
ClockWise
Sending command: cw 18.0
Response: b'ok'
newValueY= 106.0
ClockWise
Sending command: cw 16.0
Response: b'ok'
newValueY= 110.0
ClockWise
Sending command: cw 20.0
Response: b'ok'
newValueX= 128.0
forward
Sending command: forward 90
Response: b'ok'
newValueX= 67.0
backward
Sending command: back 90
Response: b'ok'
newValueX= 145.0
forward
Sending command: forward 90
Response: b'ok'
newValueX= 55.0
backward
Sending command: back 90
Response: b'ok'
newValueY= 41.0
AntiClockWise
Sending command: ccw 49.0
Response: b'ok'
newValueY= 109.0
ClockWise
Sending command: cw 19.0
Response: b'ok'
newValueY= 110.0
ClockWise
Sending command: cw 20.0
Response: b'ok

OSError: [WinError 10049] The requested address is not valid in its context

In [ ]:
telloDrone.land()